In [1]:
import sys
sys.path.append('/home/jra21/work/source/repos/opep')

In [2]:
from openep import io as openep_io
from openep import case as openep_case
from openep import mesh_routines as openep_mesh
from openep import case_routines as case_routines

import pyvista as pv
import numpy as np


In [4]:
filename = '/home/jra21/work/source/repos/opep/examples/data/new_dataset_2.mat'

In [4]:

ep_case = openep_io.load_case(filename)
ep_case_mesh = ep_case.create_mesh()
pts = ep_case.nodes
tri = ep_case.indices.astype(int)




In [67]:
import trimesh as tm

# Freeboundary
tm_mesh = tm.Trimesh(vertices=pts,faces=tri)
freeboundary = tm_mesh.outline().to_dict()
mesh_outline_entities = tm_mesh.outline().entities

freeboundary_vertex_nodes = []
freeboundary_points = []


for i in range(len(mesh_outline_entities)):
    freeboundary_vertex_nodes.append(freeboundary["entities"][i]['points'])

# print(freeboundary_vertex_nodes)

for item in freeboundary_vertex_nodes:
    print(pts[item])

[[-41.748  21.007  75.722]
 [ 16.942   8.918 116.   ]
 [-49.568  10.886 109.576]
 [ 49.46   -3.486  92.849]
 [-26.881  18.192  88.574]
 [ 19.833  -9.023 115.091]
 [ 17.567  10.734 115.035]
 [ 40.581  15.146 112.396]
 [ 17.055  27.444 111.436]
 [ 49.913  -1.648  90.032]
 [-23.334  -0.112 113.51 ]
 [ 32.873  44.614 105.543]
 [  6.702 -15.356 110.148]
 [-24.606   0.267 103.74 ]
 [-16.747  23.35   94.862]
 [ 11.882 -21.949  90.644]
 [ -0.611  11.711  78.617]
 [-41.92   19.959  98.679]
 [ 49.569  32.038  86.976]
 [-19.883  -5.445 116.577]
 [ 39.958  44.673  94.592]
 [ 34.239  33.284  85.399]
 [ -3.96   24.528 106.787]
 [ 49.349  53.989  99.931]
 [-32.391  22.852 116.719]
 [-20.837  -9.414 109.685]
 [ 33.339  -9.765 115.927]
 [ 50.658  54.077 103.352]
 [ 32.651  23.054  77.709]
 [ 35.342   7.474 114.949]
 [ 37.117  22.377 103.987]
 [-38.82   15.328  73.758]
 [ 23.25  -21.931  97.925]
 [ 19.707  -4.219 115.101]
 [-45.489  31.429 117.968]
 [  6.297   9.104 121.165]
 [ 11.425  11.102  80.584]
 

In [4]:
# mesh = pv.PolyData(pts,tri)


# mesh points
vertices = np.array([[0, 0, 0],
                     [1, 0, 0],
                     [1, 1, 0],
                     [0, 1, 0],
                     [0.5, 0.5, -1]])

# mesh faces
faces = np.hstack([[4, 0, 1, 2, 3],  # square
                   [3, 0, 1, 4],     # triangle
                   [3, 1, 2, 4]])    # triangle

surf = pv.PolyData(vertices, faces)
surf.plot(scalars=np.arange(3), cpos=[-1, 1, 0.5])


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [56]:
np.set_printoptions(suppress=True)
size_tri = []

x = tri[:,0].reshape(len(tri[:,0]),1)
y = tri[:,1].reshape(len(tri[:,1]),1)
z = tri[:,2].reshape(len(tri[:,2]),1)


for item in tri:
    size_tri.append(len(item))

# size_tri = np.array(size_tri).reshape(len(size_tri),1)

size_tri_list = size_tri
size_tri_arr = np.array(size_tri,dtype=np.int).reshape(len(size_tri),1)

# print('x.shape',x)
# print('y.shape',y)
# print('z.shape',z)
# print('size-tri.shape',size_tri_arr)

face = np.hstack(np.concatenate((size_tri_arr,tri),axis=1))
print(face)

mesh = pv.PolyData(pts,face)
mesh.plot()


[   3  533 2928 ... 9272 9221 8698]


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [51]:
faces = np.hstack([[4, 0, 1, 2, 3],  # quad
                       [3, 0, 1, 4],     # triangle
                       [3, 1, 2, 4],     # triangle
                       [1, 5]]).astype(np.int8)  # vertex
print(faces)

[4 0 1 2 3 3 0 1 4 3 1 2 4 1 5]


In [4]:
filename = '/home/jra21/work/source/repos/opep/examples/data/new_dataset_2.mat'
ep_case = openep_io.load_case(filename)

### Get WIndow Of Interest

In [10]:
def getWindowOfInterest(mesh_case,*args):

# Usage:
# woi = getWindowOfInterest(userdata) - returns the entire array of woi values from the case dataset
# woi = getWindowOfInterest(userdata, 'iEgm', 1) - returns the 1st woi value
# woi = getWindowOfInterest(userdata, 'iEgm', [100,150]) - returns the 100th and 150th woi value
# woi = getWindowOfInterest(userdata, 'iEgm', range(12,20)) - returns a range of woi values starting from 12th to 19th
    
    nStandardArgs = 1
    iEgm = np.array(':')
    
    nargin = len(args)+1
    if nargin>nStandardArgs:
        for i in range(0,(nargin-nStandardArgs),2):
            if np.char.lower(args[i]) == 'iegm':
                iEgm = args[i+1]
    woi = []
    if (type(iEgm)==str) and (np.char.compare_chararrays(iEgm,':','==',True)):
        woi = mesh_case.electric['annotations/woi'].T
        woi = woi.astype(int)
        
    else:
        woi_raw = ep_case.electric['annotations/woi'].T
        woi_raw = woi_raw.astype(int)
        for i in iEgm:
            woi.append(woi_raw[i])
    
    woi = np.array(woi)
    return woi
    

In [12]:
iEgm_range = range(12,20)

c = getWindowOfInterest(ep_case,'iEgm',[12,20])
print(c)

[[ 20 180]
 [ 20 180]]


### UseCase09: GetAnatomicalStructure()

In [193]:
import trimesh as tm
from scipy.spatial import Delaunay

def getAnatomicalStructures(ep_case, *args):
    
    
    x_values = []
    y_values = []
    z_values = []

    tri = []
    a = []
    dist = []
    l = []
    
    pts = ep_case.nodes
    face = ep_case.indices.astype(int)
        
    tm_mesh = tm.Trimesh(vertices=pts,faces=face)
    freeboundary = tm_mesh.outline().to_dict()
    mesh_outline_entities = tm_mesh.outline().entities
    freeboundary_vertex_nodes = []
    freeboundary_points = []


    for i in range(len(mesh_outline_entities)):
        freeboundary_vertex_nodes.append(freeboundary["entities"][i]['points'])


    # mesh Vertices
    trimesh_points = tm_mesh.vertices

    for i in freeboundary_vertex_nodes:
        x_values.append(trimesh_points[i][:,0])
        y_values.append(trimesh_points[i][:,1])
        z_values.append(trimesh_points[i][:,2])

    x_values_array = []
    y_values_array = []
    z_values_array = []


    for i in range(len(mesh_outline_entities)):
        x_values_array.append(x_values[i].reshape(len(x_values[i]),1))
        y_values_array.append(y_values[i].reshape(len(y_values[i]),1))
        z_values_array.append(z_values[i].reshape(len(z_values[i]),1))
        freeboundary_points.append(np.concatenate((x_values_array[i],
                                            y_values_array[i],
                                            z_values_array[i]),
                                            axis=1))

    freeboundary_points = np.array(freeboundary_points,dtype=object)


    for i in range(len(freeboundary_points)):
        coords = freeboundary_points[i]
        centre = np.round(np.mean(coords,0),3)
        centre = centre.reshape(1,len(centre))

        # Create a Trirep of the boundary
        X = np.vstack((centre,coords))
        numpts = X.shape[0]
        A = np.zeros(((numpts-1),1),dtype=np.int64)
        B = np.array(range(1,numpts)).T
        B = B.reshape(len(B),1)
        C = np.array(range(2,numpts)).T
        C = C.reshape(len(C),1)
        C = np.vstack((C,1))
        TRI = np.concatenate((A,B,C),axis=1)

        for j in range(len(coords)-1):
            dist.append(np.linalg.norm(np.subtract(coords[j+1],coords[j])))

        tri.append(tm.Trimesh(vertices=X,faces=TRI))
        a.append(round(tri[i].area,4))
        l.append(round(np.sum(dist),4))
        print('Periemeter is :',l[i],'| Area is:',a[i])


    return freeboundary_points

In [195]:
fbp = getAnatomicalStructures(ep_case)

Periemeter is : 49.6852 | Area is: 154.7759
Periemeter is : 148.6109 | Area is: 634.7572
Periemeter is : 218.0712 | Area is: 331.5392
Periemeter is : 284.9198 | Area is: 260.1277
Periemeter is : 350.7195 | Area is: 282.0847
Periemeter is : 472.9734 | Area is: 949.7377
Periemeter is : 557.916 | Area is: 508.7818
